In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def get_compiled_model():
    inputs=keras.Input(shape=(784,))
    x=keras.layers.Dense(256,activation="relu")(inputs)
    x=keras.layers.Dense(256,activation="relu")(x)
    outputs=keras.layers.Dense(10)(x)
    model=keras.Model(inputs,outputs)
    model.compile(
         optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model
def get_dataset():
    batch_size =32
    num_val_samples =500
    (x_train,y_train),(x_test,y_test)= keras.datasets.mnist.load_data()
    x_train=x_train.reshape(-1,784).astype("float32")/255
    x_test=x_test.reshape(-1,784).astype("float32")/255
    y_train=y_train.astype("float32")
    y_test=y_test.astype("float32")
    x_val=x_train[-num_val_samples:]
    y_val=y_train[-num_val_samples:]
    x_train=x_train[:-num_val_samples]
    y_train=y_train[:-num_val_samples]
    return(
        tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_val,y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(batch_size),
    )
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))
with strategy.scope():
    model=get_compiled_model()
train_dataset, val_dataset,test_dataset=get_dataset()

history=model.fit(train_dataset, epochs=100,validation_data=val_dataset)
model.evaluate(test_dataset)